In [ ]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.get_all_study_names()

In [ ]:
study_name = "trend_example"
df = optimizer.get_study_trials_df(study_name)
df

In [ ]:
df_filtered = df[(df["value"] > 1) & (df["total_executors"] > 400)].sort_values("total_executors", ascending=False)
df_filtered

In [ ]:
import plotly.express as px

px.scatter(df_filtered, x="total_executors", y="sharpe_ratio", color="net_pnl", hover_data=["number"])

In [ ]:
import json

top_configs = [json.loads(config) for config in df_filtered["config"].iloc[:5].to_list()]
top_configs

In [ ]:
for config in top_configs:
    await client.add_controller_config(config)

In [ ]:
await client.deploy_script_with_controllers(
    bot_name="test_1",
    controller_configs=[config["id"] + ".yml" for config in top_configs],
    image_name="hummingbot/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)